<!-- dom:TITLE: Exercises from Linear algebra, signal processing, and wavelets. A unified approach.\\ Python version -->
# Exercises from Linear algebra, signal processing, and wavelets. A unified approach.\\ Python version
<!-- dom:AUTHOR: Øyvind Ryan -->
<!-- Author: --> **Øyvind Ryan**

Date: **Oct 24, 2018**

<!-- Externaldocuments: applinalg -->
<!-- Mapping from exercise labels to numbers: label2numbers = {'example:movavg': '3.30', 'exercise:computeeigenvals': '3.48', 'exercise:composefilters': '3.19', 'exercise:findsparse': '3.8', 'exercise:rader': '3.11', 'example:bass': '3.35', 'example:lowpass': '3.31', 'exercise:karplusstrongiir': '3.45', 'exercise:implconv': '3.25', 'exercise:basstreble': '3.36', 'exercise:replaceshortercirc': '3.6', 'example:treble': '3.34', 'exercise:windowingops': '3.9', 'exercise:multgroupintmodn': '3.10', 'example:plotting_simple_freqresp': '3.14', 'example:echoadd': '3.16', 'example:dropping_filter_coefficients': '3.32'} -->

# Discrete time filters









<!-- --- begin exercise --- -->

## Exercise 3.6: Replacing a sparse circular convolution with shorter ones
<div id="exercise:replaceshortercirc"></div>
<!-- keywords = filters; student -->

In [1]:
import os, sys
sys.path.append(os.path.join(os.getcwd(), 'python'))

In [2]:
%matplotlib inline

from numpy import *
import time
from mp3funcs import *
from sound import *
from forward_compress_reverse import *
import matplotlib.pyplot as plt
from scipy.fftpack import dct, idct

x, fs = audioread('sounds/castanets.wav')

Let $\boldsymbol{s},\boldsymbol{x}\in\mathbb{R}^N$, and assume that $\boldsymbol{s}$ has at most $k$ nonzero entries, gathered in one segment.


**a)**
Show that there exists a $\tilde{\boldsymbol{s}}\in\mathbb{R}^{M+k-1}$ and a number $a$ so that, for any $r$,

$$
(\boldsymbol{s}\circledast\boldsymbol{x})_{r,r+1...,r+M-1} = (\tilde{\boldsymbol{s}}\circledast \tilde{\boldsymbol{x}})_{0,...,M-1},
$$

where

$$
\tilde{\boldsymbol{x}} = (x_{a+r},x_{a+r+1},...,x_{a+r+M+k-2}).
$$

In other words, any set of $M$ consecutive entries in $\boldsymbol{s}\circledast\boldsymbol{x}$ can be obtained from a circular convolution of size $M+k-1$. 
Thus, a circular convolution of size $N$ can be computed as $\frac{N}{M}$ circular convolutions of size $M+k-1$.


<!-- --- begin solution of exercise --- -->
**Solution.**
Denote by $S$ the $N\times N$ circulant Toeplitz matrix with first column equal to $\boldsymbol{s}$.
Assume that the nonzero entries in the first row of $S$ occur in the interval $[a,a+k-1]$.
Then the nonzero entries in rows $r,...,r+M-1$ of $S$ are in the columns from $a+r$ to $a+r+M-1+k-1=a+r+M+k-2$, i.e. the interval $a+[r,r+M+k-2]$ (there are $M+k-1$ such columns). With

$$
\tilde{\boldsymbol{x}} = (x_{a+r},x_{r+1},...,x_{a+r+M+k-2}),
$$

and viewing the submatrix of $S$ from rows $r,...,r+M-1$ and columns from $a+[r,r+M+k-2]$ as the first $M$ rows of a $(M+k-1)\times(M+k-1)$-circulant Toeplitz matrix $\tilde{S}$ with first column $\tilde{\boldsymbol{s}}$, we obtain immediately  that

$$
(\boldsymbol{s}\circledast\boldsymbol{x})_{r,r+1...,r+M-1} = (\tilde{\boldsymbol{s}}\circledast \tilde{\boldsymbol{x}})_{0,...,M-1}.
$$

<!-- --- end solution of exercise --- -->

**b)**
It is natural to implement the shorter circular convolutions in terms of the FFT. 
Use this strategy with $M+k-1=2^r$ to find the number of operations needed to compute the $\frac{N}{M}$ circular convolutions from a), 
and write a program which finds the $r$ which gives the lowest number of operations. 
Create a plot where $k$ is plotted against the number of operations corresponding to the optimal $r$, 
and also against the number of operations required by a direct implementation of circular convolution.


<!-- --- begin solution of exercise --- -->
**Solution.**
Recall that an $N$-point split-radix FFT requires $4N\log_2N-6N+8$ arithmetic operations. 
With $M+k-1=2^r$ for $N$ this gives $(4r-6)2^r+8$ operations for each of the shorter convolutions. 
A $2^r$-point convolution implemented in terms of the FFT (or rather FFT, multiplication in frequency, and IFFT) thus requires 

\[
2((4r-6)2^r+8) + 6\cdot 2^r = (8r-6)2^r+16
\] 
operations. The factor $2$ comes from computing both an FFT and an IFFT, and the summand $6\cdot 2^r$ comes
from the multiplications in frequency (each complex multiplication requires 4 real multiplications and 2 real additions, a total of 6 operations 
for each of the $2^r$ entries). 
In total for all the $N/M$ (or rather $\lceil N/M\rceil$) shorter convolutions we thus need $((8r-6)2^r+16)\left\lceil\frac{N}{2^r-k+1}\right\rceil$ 
operations. In a direct implementation, for each of the $N$ entries we need to make $k$ complex multiplications and $k-1$ complex additions. 
Since each complex addition requires $2$ real additions, this totals $6k+2(k-1)=8k-2$ operations for each entry, so that the overall total 
is $(8k-2)N$ operations. 

The following code finds the $r$ which gives the minimum of $((8r-6)2^r+16)/(2^r-k+1)$ for a fixed $k$, 
and then stores that minimum value. This is then repeated for all the $k$, and compared with $8k-2$, as we deduced for a direct implementation.

In [3]:
kvals = arange(1, 51)
opt_vals = kvals.copy()
for k in kvals:
    rvals = arange(ceil(log2(k)),50)
    num_ops = ((8*rvals-6)*2**rvals + 16)/(2**rvals-k+1)
    opt_vals[k-1] = num_ops.min()
plt.plot(kvals, 8*kvals-2,kvals,opt_vals)

Although this code does not plot the optimal value for $r$, this value is very much needed as a guideline for implementations on how to split 
convolutions into shorter convolutions.
The reason that we iterate $r$ from $\log_2 k$ in the code is that $k \leq 2^r$ secures that there is room for all the filter coefficients inside the vector $\tilde{\boldsymbol{s}}$ used in the smaller convolution. 
From the resulting plot we see that when the number of filter coefficients gets above $5$, the FFT-based approach works better. 
When the number of filter coefficients is below this, a direct implementation of the filter should be chosen. 

When the number of points in an FFT gets high, one usually not cares about the terms $-6N+8$ from the exact operation count of the Split radix FFT, 
since they are not dominant terms. 
In this exercise these terms really mattered, however, since short FFT's are in focus. 
If you had dropped the terms $-6N+8$, you can verify that $((8r-6)2^r+16)/(2^r-k+1)$ should be replaced by $(8r+6)2^r/(2^r-k+1)$,
and that a new plot shows that at least 8 filter coefficients are needed in order for the FFT-based approach to outperform the direct implementation.

<!-- --- end solution of exercise --- -->

<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Exercise 3.8: Finding sparse solutions to under-determined linear systems
<div id="exercise:findsparse"></div>
<!-- keywords = filters -->

Let $\boldsymbol{x}\in\mathbb{C}^N$ be a vector where we know that at most $s$ components are nonzero. When $s$ is small such vectors are called *sparse*.
Denote the indices of these nonzero components by $S$, and let $\boldsymbol{y}=DFT_N\boldsymbol{x}$. 
In this exercise we will find a procedure for finding $\boldsymbol{x}$ from the values $y_0,...,y_{2s-1}$, i.e. we can recover all components of a sparse vector from an under-determined set of measurements. 
This procedure is also called a *Reed-Solomon code*. 
In some sense this parallels the sampling theorem (which in a similar way fills in the values between the samples), with an assumption on the highest frequency replaced by an assumption on sparsity.
Many other results also exist on how to recover sparse vectors, and these results give rise to the field of *compressed sensing*. See [[foucart]](#foucart) for a review on this field.


**a)**
Let $\boldsymbol{z}$ be the vector with components $z_k=\frac{1}{N}\prod_{n\in S} \left(1-e^{-2\pi in/N}e^{2\pi ik/N}\right)$. 
Show that $z_k=0$ for $k\in S$, and also that $\boldsymbol{x}\circ\boldsymbol{z}=\boldsymbol{0}$.


<!-- --- begin solution of exercise --- -->
**Solution.**
This follows from that $1-e^{-2\pi ik/N}e^{2\pi ik/N}=1-1=0$ is a factor in $z_k$ when $k\in S$. 
Since $x_k=0$ when $k\not\in S$, and $z_k=0$ when $k\in S$, we have for all $k$ that  $(\boldsymbol{x}\circ\boldsymbol{z})_k=x_kz_k=0$.

<!-- --- end solution of exercise --- -->

**b)**
Let $\boldsymbol{w}=DFT_N\boldsymbol{z}$. Show that $w_0=1$ and that $w_n=0$ for $n>s$. 
Conclude from this, that $\boldsymbol{x}\circ\boldsymbol{z}=\boldsymbol{0}$, and due the previous exercise that $w_1,w_2,...,w_s$ 
must fulfill the equations

$$
\begin{array}{ccccccccc}
y_s      &+& w_1y_{s-1}   &+& \cdots &+& w_sy_0     &=& 0 \\ 
y_{s+1}  &+& w_1y_s       &+& \cdots &+& w_sy_1     &=& 0 \\ 
\vdots   & & \vdots       & & \ddots & & \vdots     &=& 0 \\ 
y_{2s-1} &+& w_1y_{2s-2}  &+& \cdots &+& w_sy_{s-1} &=& 0
\end{array}
$$

<!-- --- begin solution of exercise --- -->
**Solution.**
If we multiply out the factors in $z_k$ we obtain $1+\sum_{r=1}^{s} c_r e^{2\pi irk}$ for some values $c_r$. 
Since also $\boldsymbol{z}=\sum_{n=1}^N w_ne^{2\pi ikn/N}$ by the definition of the DFT, we see that $w_0=1$ and that 
$w_n=0$ for $n>s$. From the previous exercise, multiplication in time corresponds to convolution in frequency, 
so that $\boldsymbol{y}\circledast\boldsymbol{w}=\boldsymbol{0}$. The stated equation system corresponds to

$$
(\boldsymbol{y}\circledast\boldsymbol{w})_s=\cdots=(\boldsymbol{y}\circledast\boldsymbol{w})_{2s-1}=0.
$$

where we inserted $w_0=1$, $w_n=0$ for $n>s$.

<!-- --- end solution of exercise --- -->

We now have $s$ equations with $s$ unknowns. By construction this has a solution, 
but there may not be a unique solution. In the last part of the exercise we will show how to find $\boldsymbol{x}$, 
regardless of which solution we choose. 

**c)**
Assume that $v_1,v_2,...,v_s$ is any solution to the system in b), and extend this to a vector $\boldsymbol{v}\in\mathbb{R}^N$ by setting $v_0=1$, $v_{s+1}=\cdots=v_{N-1}=0$. 
Show that $(\boldsymbol{v}\circledast\boldsymbol{y})_n=0$ for $s\leq n\leq 2s-1$, and conclude from the previous exercise that 
$\left(DFT_N\left((IDFT_N\boldsymbol{v})\circ\boldsymbol{x}\right)\right)_n=0$ for $s\leq n\leq 2s-1$.


<!-- --- begin solution of exercise --- -->
**Solution.**
For $s\leq n\leq 2s-1$ we have that

$$
(\boldsymbol{v}\circledast\boldsymbol{y})_n = \sum_{k=0}^{N-1} v_k y_{(n-k)\bmod N} = y_n + \sum_{k=1}^{s} v_k y_{(n-k)\bmod N} = 0.
$$

Since $DFT_N\left((IDFT_N\boldsymbol{v})\circ\boldsymbol{x}\right) = \frac{1}{N}\boldsymbol{v}\circledast\boldsymbol{y}$ from the previous exercise, 
the result follows.

<!-- --- end solution of exercise --- -->

The vector $(IDFT_N\boldsymbol{v})\circ\boldsymbol{x}$ has at most $s$ nonzero components (since $\boldsymbol{x}$ has). 
If we take the columns from $DFT_N$ with indices in $S$, and the rows with indices between $s$ and $2s-1$,
the resulting $s\times s$-matrix was shown in Exercise 2.17 to be non-singular. 


**d)**
Explain that $((IDFT_N\boldsymbol{v})\circ\boldsymbol{x})_n=0$ for $n\in S$, and conclude that $(IDFT_N\boldsymbol{v})\circ\boldsymbol{x}=\boldsymbol{0}$.


<!-- --- begin solution of exercise --- -->
**Solution.**
The stated non-singular matrix applied to the components of $(IDFT_N\boldsymbol{v})\circ\boldsymbol{x}$ from $S$ produces zero. 
These components must thus also be zero, so that $(IDFT_N\boldsymbol{v})\circ\boldsymbol{x}$ have all components in $S$ equal to $0$. Since $\boldsymbol{x}$ is zero outside $S$, 
it follows that $(IDFT_N\boldsymbol{v})\circ\boldsymbol{x}=\boldsymbol{0}$.

<!-- --- end solution of exercise --- -->

**e)**
In particular we must have that $(IDFT_N\boldsymbol{v})_k=0$ for $k\in S$, so that

$$
\frac{1}{N}\sum_{n=0}^{N-1} v_ne^{2\pi ikn/N} = \frac{1}{N}\sum_{n=0}^s v_ne^{2\pi ikn/N} = 0
$$

for $k\in S$. This is a polynomial in $e^{2\pi ik/N}$ of degree at most $s$. Explain how one can find the set $S$ from this polynomial.


<!-- --- begin solution of exercise --- -->
**Solution.**
Simply apply an IDFT, and choose $S$ as the set from the result with smallest coefficients. Due to roundoff errors 
the coefficients will not be exactly zero, however.

<!-- --- end solution of exercise --- -->

**f)**
Explain how you can recover $\boldsymbol{x}$ from $y_0,...,y_{2s-1}$ when the set $S$ is known.


<!-- --- begin solution of exercise --- -->
**Solution.**
The submatrix $A$ from $\text{DFT}_N$ with columns from $S$, and rows from $0,...,s-1$ is again non-singular. 
The values of $\boldsymbol{x}$ on $S$, denoted $\boldsymbol{x}_s$ can be found by solving $A\boldsymbol{x}_S = (y_0,...,y_{s-1})$.

<!-- --- end solution of exercise --- -->

**g)**
Write a function

        recover_x(y, N)


which uses the procedure obtained by combining b), e), and f) to compute and return $\boldsymbol{x}\in\mathbb{R}^N$ under the assumption that $\boldsymbol{x}$ has at most $s$ nonzero components, 
and that `y` contains the first $2s$ components in $\boldsymbol{y}=DFT_N\boldsymbol{x}$. 
Test that the code works correctly on a vector $\boldsymbol{x}$ of your own choosing.


<!-- --- begin solution of exercise --- -->
**Solution.**
The function can be implemented as follows.

In [4]:
def recover_x(y, N):
    s = int(shape(y)[0]/2)
    y = reshape(y, ((2*s, 1)))
    A = zeros((s,s)).astype(complex)
    for k in range(s):
        A[:, s-k-1] = y[k:(k+s),0]
    b = -y[s:(2*s)]
    w = linalg.solve(A, b)
    v = vstack(( [[1]], w, zeros((N-(s+1),1)) ))
    
    inds = where( abs(fft.ifft(v, axis=0)) <= 0.00001 )[0]
    inds = transpose(inds.reshape( (len(inds), 1) ))
    slist = matrix(arange(0,s).reshape((s, 1)))
    x_S = linalg.solve( exp(-2*pi*1j*slist*inds/N), y[:s])
    x = zeros((N,1))
    x[inds] = x_S
    return x

The following code test that the function manages to recover a sparse vector with only four nonzero integer entries, from the $8$ first Fourier coefficients.

In [5]:
def recover_x(y, N):
    s = int(shape(y)[0]/2)
    y = reshape(y, ((2*s, 1)))
    A = zeros((s,s)).astype(complex)
    for k in range(s):
        A[:, s-k-1] = y[k:(k+s),0]
    b = -y[s:(2*s)]
    w = linalg.solve(A, b)
    v = vstack(( [[1]], w, zeros((N-(s+1),1)) ))
    
    inds = where( abs(fft.ifft(v, axis=0)) <= 0.00001 )[0]
    inds = transpose(inds.reshape( (len(inds), 1) ))
    slist = matrix(arange(0,s).reshape((s, 1)))
    x_S = linalg.solve( exp(-2*pi*1j*slist*inds/N), y[:s])
    x = zeros((N,1))
    x[inds] = x_S
    return x

<!-- --- end solution of exercise --- -->

<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Exercise 3.9: Windows
<div id="exercise:windowingops"></div>
<!-- keywords = filters -->

We mentioned in Section 2.2 that we obtain some undesirable effects in the frequency representation when we restrict to a block of the signal. Assume that $\boldsymbol{x}\in\mathbb{R}^M$, and that

$$
\boldsymbol{w}=\{\underline{w_0},\ldots,w_{N-1}\} \text{ with } N < M.
$$

We call $(w_0x_0,\ldots,w_{N-1}x_{N-1},0,...,0)\in\mathbb{R}^M$ a *windowed signal* and $\boldsymbol{w}\in\mathbb{R}^N$ a *window* of length $N$.


**a)**
Use Exercise 3.7 to show that the DFT of the windowed signal is

$$
\begin{align*}
\frac{1}{M}(DFT_M(w_0,...,w_{N-1},0,...,0))\circledast (DFT_M\boldsymbol{x})
\end{align*}
$$

<!-- --- begin solution of exercise --- -->
**Solution.**
With $\boldsymbol{s}_1=(w_0,...,w_{N-1},0,...,0)$, $\boldsymbol{s}_2=\boldsymbol{x}$ we have that $\boldsymbol{s}_1 \circ \boldsymbol{s}_2$ is the windowed signal. The result now follows from the second property in Exercise 3.7.

<!-- --- end solution of exercise --- -->

If $DFT_M(w_0,w_1,...,w_{N-1},0,...,0)$ is close to $(M,0,...,0)$, this will be close to $\boldsymbol{y}=DFT_M\boldsymbol{x}$. 
In other words, a good window should satisfy

$$
DFT_M(w_0,w_1,...,w_{N-1},0,...,0)\approx(M,0,...,0).
$$

There is a loss when we pass from the signal to the windowed signal, since we can only construct a DFT close to, not equal to, $(M,0,...,0)$. 
We will not go into techniques for how to find values $w_i$ which are close to satisfying this, only evaluate three such in the rest of the exercise. 



**b)**
The *rectangular window* is defined by $\boldsymbol{w}=\{\underline{1},1,\ldots,1\}$. Show that

$$
(DFT_M(1,...,1,0,...,0))_n = \sum_{k=0}^{N-1} e^{-2\pi ikn/M}=\frac{1-e^{-2\pi inN/M}}{1-e^{-2\pi in/M}},
$$

and use this to check whether $DFT_M(w_0,w_1,...,w_{N-1},0,...,0)\approx(M,0,...,0)$


<!-- --- begin solution of exercise --- -->
**Solution.**
This has absolute value $\left|\frac{\sin(\pi nN/M)}{\sin(\pi n/M)}\right|$. The $y_n$ thus lie on the curve 
$\frac{\sin(N\omega/2)}{\sin(\omega/2)}$, from which one can see that $DFT_M(w_0,w_1,...,w_{N-1},0,...,0)\approx(M,0,...,0)$ 
is partially fulfilled. The frequency response can be plotted as follows

In [6]:
N=32
omega = linspace(0, 2*pi, 1000)
wd = concatenate( (ones(N), zeros(1000-N)) )
plt.plot(omega, abs(fft.fft(wd)), 'k-')

<!-- --- end solution of exercise --- -->


**c)**
The *Hamming window* is defined by

$$
w_n=2(0.54-0.46\cos(2\pi n/(N-1))).
$$

Make a plot of $DFT_M(w_0,w_1,...,w_{N-1},0,...,0)$, and compare with $(M,0,...,0)$ to see if the Hamming window is a good window.


<!-- --- begin solution of exercise --- -->
**Solution.**
The frequency response of the Hamming window can be plotted as follows

In [7]:
n = arange(N)
wd = concatenate( ( 2*(0.54-0.46*cos(2*pi*n/(N-1))), zeros(1000-N)) )
plt.plot(omega, abs(fft.fft(wd)), 'k-')

The Hamming window is seen to differ from the rectangular window in two ways: 
* It has much smaller values away from $0$,

* the width of the "main lobe" (i.e. the main structure at the center) is bigger. 

As a consequence, windowing with the Hamming window reduces the contribution from higher frequencies, but increases the contribution from the smallest frequencies. 
The window coefficients for both windows 
can be plotted with the following code

In [8]:
N = 32
for k in range(N):
    plt.plot([k, k],[0, 1],'k-')

In [9]:
for k in range(N):
    plt.plot([k, k],[0, 2*(0.54-0.46*cos(2*pi*k/(N-1)))], 'k-')

<!-- --- end solution of exercise --- -->


**d)**
The *Hanning window* is defined by

$$
w_n=1-\cos(2\pi n/(N-1)).
$$

Repeat the analysis you did above for the rectangular and Hamming  windows, for a Hanning window for $N=32$. 

The Hanning window is used in the MP3 standard, where it is applied to blocks which overlap, contrary to the non-overlapping block pattern we have used.
After the Hanning window has been applied, the MP3 standard applies an FFT to the windowed signal in order to make a frequency analysis of that part of the sound.


<!-- --- begin solution of exercise --- -->
**Solution.**
The frequency response can be plotted as follows

In [10]:
wd = concatenate( ( 1-cos(2*pi*n/(N-1)), zeros(1000-N) ) )
plt.plot(omega, abs(fft.fft(wd)), 'k-')

<!-- --- end solution of exercise --- -->

A good survey on the windows mentioned here, as well as other ones, can be found in [[harris01]](#harris01). 
One lesson to be learned from this exercise is that the windowed signal may not give a good frequency representation. 
One must therefore be careful when splitting a sound into blocks, as this alters the frequency representation. 
You can use the function `forw_comp_rev_DFT` from Example 2.5 to experiment with this.
This function accepts a named parameter $N$, which can be used to split the DFT of a sound into blocks of length $N$, and eliminate low frequencies before taking an IDFT to reconstruct and listen to the new sound 
(see also Example 3.31).

<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Exercise 3.10: The multiplicative group of integers modulo $N$
<div id="exercise:multgroupintmodn"></div>
<!-- keywords = filters -->

In number theory it is known that, when $N$ is a prime number, the numbers $1,...,N-1$ is a *group* under multiplication modulo $N$. 
In particular this means that for each $1\leq a\leq N-1$ there exists a unique $1\leq b\leq N-1$ so that $ab=1\bmod N$ 
(the number $1$ acts as a *unit*). 
$b$ us then called the inverse of $a$, and we write $b=a^{-1}$ (this is not the same as $1/a$!). 
Also, it is known that there exists an integer $g$ so that the set $\{g^q\bmod N\}_{q=0}^{N-2}$ constitutes all numbers $1,...,N-1$ 
(the order these numbers appear in may have changed, though).
$g$ is called a *generator* for the group.


**a)**
Find generators for the multiplicative group of integers modulo $11$, $23$ and $41$. Are the generators for each group unique?


<!-- --- begin solution of exercise --- -->
**Solution.**
Even for small primes, one saves tedious computation here with an implementation. The following code returns the smallest generator.

In [11]:
def find_smallest_generator(N):
    for k in range(2, N):
        res = zeros(N-1)
        val = 1
        for s in range(1, N):
            if res[val-1] != 0:
                break
            res[val-1] = s
            val = mod(val*k,N)
            if s == N-1:
                return k
    return 0

The outer loop starts at $2$, because the unit $1$ is never a generator. If your run this function for the stated primes,

In [12]:
print(find_smallest_generator(11))
print(find_smallest_generator(23))
print(find_smallest_generator(41))

the generators $2$, $5$, and $6$, respectively,  are found. 

A generator may not be unique: In the next exercise we state that $g^{-1}$ is a generator whenever $g$ is.

<!-- --- end solution of exercise --- -->

**b)**
Write a function

        reorder(x, g)


which, for a vector $\boldsymbol{x}$ of length $N$, and a generator $g$ for the numbers $1,...,N-1$, 
returns the vector $(x_{g^0},x_{g^1},...,x_{g^{N-2}})$.


In the next exercise you will see how the multiplicative group of integers modulo $N$ relates to circular convolution.


<!-- --- begin solution of exercise --- -->
**Solution.**
The following code can be used.

In [13]:
def reorder(x, g):
    N = len(x)
    xnew = zeros(N-1).astype(type(x))
    val = 1
    for s in range(N-1):
        xnew[s] = x[val]
        val = mod(val*g, N)
    return xnew

<!-- --- end solution of exercise --- -->


<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Exercise 3.11: The FFT algorithm when $N$ is prime. Rader's algorithm
<div id="exercise:rader"></div>
<!-- keywords = filters -->

It is possible to show that, if $g$ is a generator, then $g^{-1}$ is also a generator.
This means that $\{g^{-p}\bmod N\}_{p=0}^{N-2}$ also constitute 
the numbers $1,...,N-1$ (where $g^{-p}$ is defined as $(g^{-1})^p$). 

Replacing $n=g^{-p}$ and $k=g^q$ in the FFT algorithm with $N$ prime, 
we can rephrase it as (we pull $k=0$ out of the sum, since the number $0$ is not an element in our group)

$$
\begin{align*}
y_0        &= \sum_{k=0}^{N-1} x_k \\ 
y_{g^{-p}} &= x_0 + \sum_{q=0}^{N-2} x_{g^q}e^{-2\pi i g^{-(p-q)}/N}    \text{, } 0\leq p\leq N-2.
\end{align*}
$$

Define

$$
\begin{align*}
\boldsymbol{a} &= (x_{g^0}        ,x_{g^1}            ,...,x_{g^{N-2}})            \in\mathbb{R}^{N-1}\\ 
\boldsymbol{b} &= (e^{-2\pi i g^0/N}, e^{-2\pi i g^{-1}/N},...,e^{-2\pi i g^{-(N-2)}/N}) \in\mathbb{R}^{N-1}\\ 
\boldsymbol{c} &= (y_{g^0}        ,y_{g^{-1}}         ,...,y_{g^{-(N-2)}})         \in\mathbb{R}^{N-1}
\end{align*}
$$

Explain that $\boldsymbol{c}=x_0+\boldsymbol{a}\circledast\boldsymbol{b}$, where $x_0$ is added to every component in the vector. 

This explains how to compute an $N$-point DFT (with $N$ prime) using ($N-1$)-point circular convolution. 
Since a circular convolution can be computed using a DFT/IDFT of the same size, so this method effectively reduces an $N$-point DFT to an $(N-1)$-point DFT. 
Since $N-1$ is not prime, we can use the algorithm for the FFT for composite $N$, to reduce the problem to DFT's of lengths being smaller prime numbers.
Another possibility is to use Exercise 3.5 to replace the circular convolution with a longer one, of length a power of two,  
effectively reducing the problem to a DFT of length a power of two.


<!-- --- begin solution of exercise --- -->
**Solution.**
Inserting the definitions of $\boldsymbol{a}$, $\boldsymbol{b}$, and $\boldsymbol{c}$ in the DFT formula above gives

$$
c_p = x_0 +  \sum_{q=0}^{N-2}  a_q b_{p-q} = x_0 +  \sum_{q=0}^{N-2}  a_q b_{(p-q)\bmod(N-1)} = x_0 + (\boldsymbol{a}\circledast\boldsymbol{b})_p.
$$

The formula follows. We used that $b_{p-q}=b_{(p-q)\bmod(N-1)}$, which follows from that $g^{N-1}$ equals the unit $e$: If this was not the case, we would have that $g^{N-1}=g^i$ for some $i>0$. This gives that $g^{N-1-i}=e$, which would imply that $g$ does not generate all numbers from $1$ to $N-1$.
The following code tests that Rader's algorithm produces the same result as an $N$-point FFT when $N$ is prime.

In [14]:
N = 41
x=random.random(N); x = x.astype(complex)
g = find_smallest_generator(N)
a = reorder(x, g)
b = reorder(exp(-2*pi*1j*(arange(0, N))/N), g); b[1:] = b[(-1):0:(-1)]

c = zeros(shape(a)).astype(complex)
for n in range(N-1):
    for k in range(N-1):
       c[n] += a[k]*b[mod(n-k,N-1)]
c += x[0] 
c[1:] = c[(-1):0:(-1)]
y = x.copy()
y[0] = sum(x)
ind = 1
for k in range(N-1):
    y[ind] = c[k]
    ind = mod(ind*g, N)
print(abs(fft.fft(x,axis=0)-y).max())

Here we   
used a direct implementation of circular convolution, since `numpy` seems to have no direct support for this.
Later we will consider our own implementation of circular convolution, but this will have a different assumption on the input. 
In the code above, also note that we reversed the vectors $\boldsymbol{b}$ and $\boldsymbol{c}$. This is because these used $g^{-1}$ rather than $g$ to generate their values.

<!-- --- end solution of exercise --- -->

<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Example 3.14: Plotting a frequency response
<div id="example:plotting_simple_freqresp"></div>
<!-- keywords = filters -->

The frequency response of the filter (3.13) is

$$
\begin{align*}
\lambda_S(\omega) &= \frac{1}{4} e^{i\omega} + \frac{1}{2}e^{0} + \frac{1}{4} e^{-i\omega} = \frac{1}{2} + \frac{1}{4} (e^{i\omega} + e^{-i\omega}) = \frac{1}{2} + \frac{1}{2}\cos(\omega).
\end{align*}
$$

We can plot the frequency response over $[0,2\pi]$ as follows,

In [15]:
omega=linspace(0, 2*pi, 100)
plt.plot(omega,0.5+0.5*cos(omega), 'k-')

and over $[-\pi,\pi]$ as follows.

In [16]:
omega=linspace(-pi, pi, 100)
plt.plot(omega,0.5+0.5*cos(omega), 'k-')

The plots clearly show that the high frequencies are made smaller by the filter, and it is therefore called a *low-pass filter*. These are investigated further in the next section. 

A more general way to plot the frequency response of a FIR filter is to
combine Proposition 7 with Theorem 2: If $\boldsymbol{t}$ is the set of filter coefficients, equation (3.3) gives that

        omega = 2*pi*arange(0,N)/float(N)
        s = concatenate([t, zeros(N - len(t))])
        plot(omega, abs(fft.fft(s))) 


Some comments are in order. First of all we have restricted to $[0,2\pi)$. Secondly, $\boldsymbol{t}$ was expanded to a longer vector $\boldsymbol{s}$, to increase the number of plot points.
Finally, the expanded vector $\boldsymbol{s}$ is correct only up to a delay, due to Proposition 7,  
and as we have seen a delay of the filter does not affect the magnitude of the frequency response.
Here only the magnitude is plotted, as we often do.

<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Example 3.16: Adding echo
<div id="example:echoadd"></div>
<!-- keywords = filters -->

In [17]:
x, fs = audioread('sounds/castanets.wav')
d = 10000
c = 0.5

An echo is a delayed and softer copy of the original sound.  If $\boldsymbol{x}$ is this original sound, the sound $\boldsymbol{z}$ with samples given by

In [18]:
N, nchannels = shape(x)
z = zeros((N, nchannels))
z[0:d] = x[0:d]                # No echo at the start
z[d:N] = x[d:N] + c*x[0:(N-d)] # Add echo
z /= abs(z).max()              # Scale to within [-1,1]

includes an echo of $\boldsymbol{x}$. 
$d$ is an integer which represents the delay in samples. If you need a delay in $t$ seconds, set $d=tf_s$ (and round to the nearest integer), with $f_s$ the sample rate.  
$c$ is called the *damping factor*. An echo is usually weaker than the original sound, so that $c < 1$. 
Let us listen to the sound after adding echo.

In [19]:
play(z, fs)

You are encouraged to experiment and find the range of $d$ which makes the echo indistinguishable from the sound itself, 
and how low can you choose $c$ in order to still hear the echo.

The compact notation for a filter which adds echo is

$$
S=\{ \underline{1},0,\ldots,0,c \},
$$

where the damping factor $c$ appears at index $d$. 
The frequency response of this is $\lambda_S(\omega)=1+ce^{-id\omega}$, which is not real. 
We can plot the frequency response as follows

In [20]:
omega = linspace(0.01, 2*pi-0.01, 100) 
plt.plot(omega, abs(1+0.1*exp(-1j*10*omega)), 'k-')

We see that the response varies between $1-c$ and $1+c$. The oscillation is controlled by the delay $d$.

<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Exercise 3.19: Composing two filters
<div id="exercise:composefilters"></div>
<!-- keywords = filters -->

Assume that the filters $S_1$ and $S_2$ have the frequency responses $\lambda_{S_1}(\omega)=2+4\cos(\omega)$, $\lambda_{S_2}(\omega)=3\sin(2\omega)$.


**a)**
Compute and plot the frequency response of the filter $S_1S_2$.


<!-- --- begin solution of exercise --- -->
**Solution.**
We have that

$$
\lambda_{S_1S_2}(\omega) = \lambda_{S_1}(\omega) \lambda_{S_2}(\omega) = (2+4\cos(\omega))(3\sin(2\omega)).
$$

The frequency response can be plotted as follows.

In [21]:
omega=linspace(-pi,pi,100)
plt.plot(omega, (2+4*cos(omega))*(3*sin(2*omega)))

<!-- --- end solution of exercise --- -->

**b)**
Write down the filter coefficients $t_k$ for the filter $S_1S_2$.


<!-- --- begin solution of exercise --- -->
**Solution.**
We have that

$$
\begin{align*}
\lambda_{S_1S_2}(\omega) &= (2+4\cos(\omega))(3\sin(2\omega)) = (2+2e^{i\omega}+2e^{-i\omega})\frac{3}{2i}(e^{2i\omega}-e^{-2i\omega}) \\ 
                         &= -3i (1+e^{i\omega}+e^{-i\omega})(e^{2i\omega}-e^{-2i\omega}) \\ 
                         &= -3i(e^{2i\omega}-e^{-2i\omega} + e^{3i\omega} + e^{i\omega} - e^{-i\omega} - e^{-3i\omega}).
\end{align*}
$$

From this we see that $S_1S_2=3i\{-1,-1,-1,\underline{0},1,1,1\}$.

<!-- --- end solution of exercise --- -->

<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Exercise 3.25: Execution times for convolution
<div id="exercise:implconv"></div>
<!-- keywords = filters; student -->

Implement code which computes $\boldsymbol{t}\ast\boldsymbol{x}$ in the vectorized- and non-vectorized ways described in Section 3.2.1 (i.e. as a single loop over $k$ or $n$ with the other variable vectorized, or a double loop). 
As your $\boldsymbol{t}$, take $k$ randomly generated numbers. Compare execution times with the `convolve` function, for different values of $k$. Present the result as a
plot where $k$ runs along the $x$-axis, and execution times run along the $y$-axis. Your result will depend on how vectorization is performed by the computing environment.





<!-- --- begin solution of exercise --- -->
**Solution.**
The code can look as follows.

In [22]:
x, fs = audioread('sounds/castanets.wav')
x = x[:,0]
N= len(x)

kmax=100
vals1 = zeros(int(kmax/10))
vals2 = zeros(int(kmax/10))
vals3 = zeros(int(kmax/10))
ind = 0
for k in range(10, kmax+1,10):
    t = random.random(k)
    start = time.time()
    convolve(t, x)
    vals1[ind] = time.time() - start
    
    z = zeros(N)
    start = time.time()
    for s in range(k):
        z[(k-1):N] += t[s]*x[(k-1-s):(N-s)]
    vals2[ind] = time.time() - start

    z = zeros(N)
    start = time.time()
    for n in range(k-1, N):
        for s in range(k):
            z[n] += t[s]*x[n-s]
    vals3[ind] = time.time() - start
    ind += 1
plt.plot( range(10, kmax+1,10), log(vals1), 'r-', \
          range(10, kmax+1,10), log(vals2), 'g-', \
          range(10, kmax+1,10), log(vals3), 'b-', )
plt.legend(['conv','simple for','double for'])

<!-- --- end solution of exercise --- -->

<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Example 3.30: Moving average filters
<div id="example:movavg"></div>
<!-- keywords = filters -->

A general way of reducing variations in sound is to replace one number by the average of itself and its neighbors. 
If $\boldsymbol{z}=(z_i)_{i=0}^{N-1}$ is the sound signal produced by taking the average of three successive samples, we have that

$$
z_n = \frac{1}{3}(x_{n+1} + x_n + x_{n-1}),
$$

i.e. $S=\{1/3,\underline{1/3},1/3\}$. This is also called a *moving average filter*  (with three elements). 
If we set $N=4$, the corresponding circulant Toeplitz matrix for the filter is

$$
S
=
\frac{1}{3}
\begin{pmatrix}
1 & 1 & 0           & 1 \\ 
1 & 1 & 1 & 0 \\ 
0           & 1 & 1 & 1 \\ 
1 & 0           & 1 & 1           
\end{pmatrix}
$$

The frequency response is

$$
\lambda_S(\omega)=(e^{i\omega}+1+e^{-i\omega})/3=(1+2\cos(\omega))/3.
$$

More generally we can construct the moving average filter of $2L+1$ elements, which is $S=\{1,\cdots,\underline{1},\cdots,1\}/(2L+1)$ (there is symmetry around $t_0$). 
Let us verify that these filters are low-pass filters.

Clearly the first column of $S$ is $\boldsymbol{s}=(\underbrace{1,\ldots,1}_{L+1\text{ times}},0,\ldots,0,\underbrace{1,\ldots,1}_{L\text{ times}})/(2L+1)$.
In Example 2.2 we computed that

$$
DFT_N\left(\underbrace{1,\ldots,1}_{L+1\text{ times}},0,\ldots,0,\underbrace{1,\ldots,1}_{L\text{ times}}\right)=\boldsymbol{y}
$$

where $\boldsymbol{y}$ had components

$$
y_n = \frac{\sin(\pi n(2L+1)/N)}{\sin(\pi n/N)}.
$$

Since $\boldsymbol{\lambda}_S=\text{DFT}_N\boldsymbol{s}$, dividing by $2L+1$ and inserting $\omega=2\pi n/N$ gives that

$$
\lambda_S(\omega)=\frac{1}{2L+1}\frac{\sin((2L+1)\omega/2)}{\sin(\omega/2)}.
$$

We clearly have

$$
0\leq\frac{1}{2L+1}\frac{\sin((2L+1)\omega/2)}{\sin(\omega/2)}\leq 1,
$$

and the frequency response approaches $1$ as $\omega\to 0$, so that it peaks at $0$. This peak gets narrower and narrower as $L$ increases. 
This filter thus "keeps" only the lowest frequencies. It is also seen that the frequency response is small for $\omega\approx\pi$. 
In fact it is straightforward to see that $|\lambda_S(\pi)|=1/(2L+1)$. 
The frequency responses for the moving average filters corresponding to $L=1$, $L=5$, and $L=20$
can be plotted as follows.

In [23]:
omega = linspace(0.01, 2*pi-0.01, 100) 
for L in [1, 5, 20]:
    plt.figure()
    plt.plot(omega, sin((2*L+1)*omega/2)/(sin(omega/2)*(2*L+1)), 'k-')

In conclusion, moving average filters are low-pass filters, but they are far from ideal such, since not all higher frequencies are annihilated, and since small frequencies also are changed.

<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Example 3.31: Ideal low-pass filters
<div id="example:lowpass"></div>
<!-- keywords = filters -->

By definition, the ideal low-pass filter keeps frequencies near $0$ unchanged, and completely removes frequencies near $\pi$. 
$S=(F_N)^HDF_N$ is an ideal low-pass filter when $D$ is diagonal with

$$
(\underbrace{1,\ldots,1}_{L+1\text{ times}},0,\ldots,0,\underbrace{1,\ldots,1}_{L\text{ times}})
$$

on the diagonal. 
If the filter should keep the angular frequencies $|\omega|\leq\omega_c$ only, where $\omega_c$ is the "cutoff" frequency, we should choose $L$ so that $\omega_c=2\pi L/N$. 
An ideal high-pass filters similarly corresponds to a diagonal matrix with

$$
(\underbrace{0,\ldots,0}_{N/2-L\text{ times}},\underbrace{1,\ldots,1}_{2L+1\text{ times}},\underbrace{0,\ldots,0}_{N/2-L-1\text{ times}})
$$

on the diagonal.

Let us compute the filter coefficients for the ideal low-pass filter.
Again, in Example 2.2 we computed the DFT of the vector above, and it followed from Theorem 7 that the IDFT of this vector equals its DFT, up to a factor $1/N$. 
This means that $\boldsymbol{s}=IDFT_N\boldsymbol{\lambda}_S$ is

$$
\frac{1}{N}\frac{\sin(\pi k(2L+1)/N)}{\sin(\pi k/N)}.
$$

The filter coefficients are thus $N$ points uniformly spaced between $0$ and $1$ on the curve $\frac{1}{N}\frac{\sin(\pi t(2L+1)/2)}{\sin(\pi t/2)}$. 
This curve has been encountered many other places in the book. 
Moving average filters and ideal low-pass filters thus have a duality between vectors which contain only zeros and ones on one side (i.e. windows), 
and the vector $\frac{1}{N}\frac{\sin(\pi k(2L+1)/N)}{\sin(\pi k/N)}$ on the other side: 
filters of the one type correspond to frequency responses of the other type, and vice versa. 

The extreme cases for $L$ are
* $L=1$: Only the lowest frequency is kept. All filter coefficients are equal to $\frac{1}{N}$. 

* $L=N$: All frequencies are kept. The filter equals the identity matrix.

Between these two extremes, $s_0$ is the biggest coefficient, while the others decrease towards $0$ along the curve we stated. 
The bigger $L$ and $N$ are, the quicker they decrease to zero. All filter coefficients are usually nonzero for this filter, since this curve is zero only at certain points. 
The filter is thus not a FIR filter. Many filters which are not FIR still have efficient implementations, but for this filter it turns out to be difficult to find one.
The best thing we can do is probably to use a DFT for computing the filter, followed by an inverse DFT.

The function `forw_comp_rev_DFT` from Example 2.5 accepts named parameters `L` and `lower`, where `L` is as described above, and where `lower` states whether the lowest or the highest frequencies should be kept. 
We can use the function to listen to the lower frequencies in the audio sample file.
We start with $L=13000$.

In [24]:
x, fs = forw_comp_rev_DFT(L=13000, lower=1)
play(x, fs)

Then with $L=5000$.

In [25]:
x, fs = forw_comp_rev_DFT(L=5000, lower=1)
play(x, fs)

With $L=13000$ you can hear the disturbance in the sound, but we have not lost that much even if about 90\% of the DFT coefficients are dropped. 
The quality is much poorer when $L=5000$ (here we keep less than 5\% of the DFT coefficients). 
However we can still recognize the song, and this suggests that most of the frequency information is contained in the lower frequencies.

Let us then listen to higher frequencies instead. 
We start with $L=140000$.

In [26]:
x, fs = forw_comp_rev_DFT(L=140000, lower=0)
play(x, fs)

Then with $L=100000$.

In [27]:
x, fs = forw_comp_rev_DFT(L=100000, lower=0)
play(x, fs)

We find that we need very high values of $L$ to hear anything, suggesting again that most information is contained in the lowest frequencies.





<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Example 3.32: Windowing an ideal low-pass filter
<div id="example:dropping_filter_coefficients"></div>
<!-- keywords = filters -->

In order to decrease the operation count of the ideal low-pass filter, one could apply a rectangular window to the filter coefficients (see Exercise 3.9), i.e. consider the filter

$$
\left\{\frac{1}{N}\frac{\sin(\pi k(2L+1)/N)}{\sin(\pi k/N)}\right\}_{k=-N_0}^{N_0}.
$$

In light of that exercise, this may not be the best strategy - applying a window different from the rectangular one may better preserve the frequency response of the ideal low-pass filter. 

Consider the ideal low-pass filter with $N=128$, $L=32$ (i.e. the filter removes frequencies $\omega>\pi/2$). 
The following code tests this for different values of $N_0$.

In [28]:
N = 128
L = 32
filtercoeffs = [sin(pi*k*(2*L+1)/N)/(sin(pi*k/N)*N) for k in range(1, N)]
omega=linspace(0, 2*pi, 100)
      
for N0 in [2, 4, 16, 64]:
    plt.figure()
    freqresp=((2*L+1)/float(N))*ones(len(omega));
    for k in range(1, N0 + 1):
        freqresp += filtercoeffs[k - 1]*2*cos(omega*k)
    plt.plot(omega, freqresp, 'k-')

This shows that we should be careful when we omit filter coefficients: if we drop too many, the frequency response is far away from that of an ideal
low-pass filter. In particular, we see that the new frequency response oscillates wildly near the discontinuity of the ideal low-pass filter. 





<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Example 3.34: Low-pass filters deduced from Pascal's triangle
<div id="example:treble"></div>
<!-- keywords = filters -->

When computing an average, it is reasonable to let the middle sample count more than the neighbors. So, an alternative to that of moving averages is to compute

$$
z_n = (x_{n-1} + 2x_n + x_{n+1})/4
$$

The coefficients $1,2,1$ are taken from row 2 in Pascal's triangle. It turns out that this is a good choice of coefficients. 
In fact, other rows from Pascal's triangle are also good choices. 
To explain why, let $S=\{\underline{1/2},1/2\}$ be the moving average filter of two elements.
The frequency response of $S$ is

$$
\lambda_S(\omega)=\frac{1}{2}(1+e^{-i\omega})=e^{-i\omega/2}\cos(\omega/2).
$$

If we apply this filter $k$ times, Theorem 6 states that

$$
\lambda_{S^k}(\omega)=\frac{1}{2^k}(1+e^{-i\omega})^k=e^{-ik\omega/2}\cos^k(\omega/2),
$$

which is a polynomial in $e^{-i\omega}$ with the coefficients taken from Pascal's triangle (the values in Pascal's triangle are the
coefficients of $x$ in the expression $(1+x)^k$, i.e. the binomial coefficients $\binom{k}{r}$ for $0\leq r\leq k$). 
Thus, the filter coefficients of $S^k$ are rows in Pascal's triangle.  
The reason why the filters $S^k$ are more desirable than moving average filters is that,
since $(1+e^{-i\omega})^k$ is a factor in its frequency response, it has a zero of multiplicity $k$ at $\omega=\pi$. 
This implies that the frequency response is very flat for $\omega\approx\pi$ when $k$ increases, i.e. the filter is good at removing the highest frequencies. This can be seen
with help from the following code, where we plot
the magnitude of the frequency response when $k=5$, and when $k=30$.

In [29]:
omega=linspace(0, 2*pi, 100)
for k in [5, 30]:
    plt.figure()
    plt.plot(omega, abs((cos(omega/2))**k), 'k-')

Clearly the latter frequency response is much flatter for $\omega\approx\pi$. On the other side, the filters of Example 3.30 satisfied $|\lambda_S(\pi)|=1/(2L+1)$, with a frequency response not very flat near $\pi$ 

While using $S^k$ gives a desirable behavior for $\omega\approx\pi$, we see that the behavior is not so desirable for small frequencies $\omega\approx 0$: Only frequencies very close to $0$ are kept
unaltered. It should be possible to produce better low-pass filters than this also, as the frequency responses of the filters in the MP3 standard suggest.

Filtering with $S^k$ can be computed by iterating the filter $\{\underline{1/2},1/2\}$ $k$ times:

In [30]:
k = 5
z = x[:, 0]
for kval in range(k):
    z = convolve(z, [1/2., 1/2.])

This code disregards the circularity of $S$, and we introduce a time delay. These issues
will, however, not be audible when we listen to the output. 
In Exercise 3.36 you will be asked to perform these steps our sample audio file.

<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Example 3.35: High-pass filters deduced from Pascal's triangle
<div id="example:bass"></div>
<!-- keywords = filters -->

If we apply Proposition 11 to the low-pass filter deduced from the fourth row in Pascal's triangle we obtain

$$
z_n = \frac{1}{16}(x_{n-2}-4x_{n-1}+6x_n-4x_{n+1}+x_{n+2})
$$

Clearly the high-pass filter arising from row $k$ in Pascal's triangle can be written as $S^k$, with $S=\frac{1}{2}\{\underline{1},-1\}$. 
In other words, we can use convolution as in the previous example to compute the output from such filters. 
In Exercise 3.36 you will be asked to apply these to the audio sample file. 
The new sound will be difficult to hear for large $k$, and we will explain why later. 

The frequency response we obtain from using row $5$ of Pascal's triangle
can be plotted as follows

In [31]:
t=arange(0,0.01,1/4400.)
vals=sin(2*pi*440*t)
arr2 = concatenate([array([-1/4400.]), array([-2/4400.]), t, array([0.01+1/4400.]), array([0.01+2/4400.])])
plt.plot(arr2,convolve(vals,array([1, -4, 6, -4, 1])/16.),'-ko')

<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Exercise 3.36: Applying low-pass- and high-pass filters deduced from Pascal's triangle to the audio sample file
<div id="exercise:basstreble"></div>
<!-- keywords = filters; student -->


**a)**
Write code where you apply the low-pass and high-pass filters in examples 3.34 and 3.35 to the audio sample file, and verify that the sounds you get are the same as in these examples.
How high must $k$ be in order for you to hear difference from the actual sound? How high can you choose $k$ and still recognize the sound at all?  
If you solved Exercise 3.28 you can also use the function `filter_impl` to perform the filtering, 
rather than using convolution (which, as mentioned, discards circularity).


<!-- --- begin solution of exercise --- -->
**Solution.**
The code can look like this, when we apply low-pass filters.

In [32]:
k = 5
z = x[:, 0]
for kval in range(k):
    z = convolve(z, [1/2., 1/2.])

In [33]:
play(z, fs)

If we apply high-pass filters instead, the code can look like this.

In [34]:
z = x[:, 0]
for kval in range(k):
    z = convolve(z, [1/2., -1/2.])

In [35]:
play(z, fs)

<!-- --- end solution of exercise --- -->

**b)**
In your code, it is not necessary to scale the values after applying the low-pass or high-pass filters so that values fit inside $[-1,1]$. Explain why this is the case.


<!-- --- begin solution of exercise --- -->
**Solution.**
The low-pass filters compute weighted averages of the input samples. Therefore, as long as the input values are inside the legal range$ [-1,1]$, the output values will as well. 
Since the filter coefficients sum to one it is easy to see that also the high-pass filters produce values in the legal range.

<!-- --- end solution of exercise --- -->

<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Exercise 3.45: The IIR filter for the Karplus-Strong algorithm
<div id="exercise:karplusstrongiir"></div>
<!-- keywords = filters -->

Let us rewrite the difference equation in Exercise 1.12 as

$$
z_{n}-\frac{1}{2}(z_{n-p}+z_{n-(p+1)}) = 0
$$

**a)**
Explain that the frequency response of this realization is 0. Therefore, this realization is only interesting if the initial conditions are chosen so that we don't obtain a filter.


<!-- --- begin solution of exercise --- -->
**Solution.**
The numerator in $\lambda_S(\omega)$ is zero, since all $b_k=0$. We therefore have that $\lambda_S(\omega)=0$ also.

<!-- --- end solution of exercise --- -->

**b)**
Plot the zeros of the characteristic equation for $p=20$. Is the filter BIBO stable?


<!-- --- begin solution of exercise --- -->
**Solution.**
We have that $a_0=1$, $a_p=a_{p+1}=-1/2$, so that the characteristing equation is $\sum_{k=0}^M a_k r^{-k} =  1  - \frac{1}{2}r^{-p} - \frac{1}{2}r^{-(p+1)}$. Its zeros can be plotted as follows.

In [36]:
p=20
rts = roots(concatenate([[1], zeros(p-1), [-1/2, -1/2]]))
plt.plot(real(rts), imag(rts), 'kx')

We see from the resulting plot that the zeros are inside the unit circle, so that the filter is BIBO stable.

<!-- --- end solution of exercise --- -->

**c)**
What happens with the zeros of the characteristic equation when you increase $p$? Attempt to explain why the resulting sound changes more slowly to a static sound when you increase $p$.


<!-- --- begin solution of exercise --- -->
**Solution.**
It seems that the absolute values of the zeros of the characteristic equation gets closer and closer to $1$ as $p$ increases. This also means that their powers decreases slower to zero, so that the resulting sound goes to zero somewhat more slowly.

<!-- --- end solution of exercise --- -->

**d)**
A related IIR filter is

$$
z_{n}-\frac{1}{2}(z_{n-p}+z_{n-(p+1)}) = x_n.
$$

Plot the frequency response of this filter for $p=20$. Include only values in the response between 0 and 10.


<!-- --- begin solution of exercise --- -->
**Solution.**
The following code can be used

In [37]:
p=20
omega = linspace(0,2*pi,500)
plt.plot(omega, abs(1/(1-0.5*exp(-omega*p*1j)-0.5*exp(-omega*(p+1)*1j))) )
plt.axis([0,2*pi,0,10])

<!-- --- end solution of exercise --- -->

<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Exercise 3.48: Computing eigenvalues
<div id="exercise:computeeigenvals"></div>
<!-- keywords = filters -->

Consider the matrix

$$
S=\frac{1}{3}\begin{pmatrix}
2 & 1 & 0 & 0 & 0 & 0 \\ 
1 & 1 & 1 & 0 & 0 & 0 \\ 
0 & 1 & 1 & 1 & 0 & 0 \\ 
0 & 0 & 1 & 1 & 1 & 0 \\ 
0 & 0 & 0 & 1 & 1 & 1 \\ 
0 & 0 & 0 & 0 & 1 & 2
\end{pmatrix}
$$

**a)**
Compute the eigenvalues and eigenvectors of $S$ using the results of this section. You should only need to perform one DFT or one DCT in order to achieve this.


<!-- --- begin solution of exercise --- -->
**Solution.**
In c) we show that this indeed corresponds to a symmetric filter. We then know that the DCT basis vectors are eigenvectors. The eigenvalues can be found on the diagonal in $\text{DCT}_NS(\text{DCT}_N)^T$. 
Note that it is not true that the eigenvalues can be computed by taking a DCT on the first column, as is the case for filters and the DFT.

<!-- --- end solution of exercise --- -->

**b)**
Use a computer to find the eigenvectors and eigenvalues of $S$ also. What are the differences from what you found in a)?


<!-- --- begin solution of exercise --- -->
**Solution.**
The following code compares  what you found in a) with a direct computation.

In [38]:
from scipy.fftpack import dct, idct

S = matrix([[2, 1, 0, 0, 0, 0],\
            [1, 1, 1, 0, 0, 0],\
            [0, 1, 1, 1, 0, 0],\
            [0, 0, 1, 1, 1, 0],\
            [0, 0, 0, 1, 1, 1],\
            [0, 0, 0, 0, 1, 2]])

idctmatr = idct(eye(6), norm='ortho', axis=0)
print(idctmatr)                              # Eigenvectors
print(dct(S*idctmatr, norm='ortho', axis=0)) # eigenvalues

w, v = linalg.eig(S)
print(w)                                     # Eigenvalues
print(v)                                     # eigenvectors

The two typically will list eigenvectors and eigenvalues in different order.

<!-- --- end solution of exercise --- -->

**c)**
Find a filter $T$ so that $S=T_r$. What kind of filter is $T$?


<!-- --- begin solution of exercise --- -->
**Solution.**
Writing down the circulant $12\times12$-matrix for the symmetric filter $T=\frac{1}{3}\{1,\underline{1},1\}$ we obtain the two upper blocks

$$
\begin{align*}
T_1 &= \frac{1}{3}
\begin{pmatrix}
1 & 1 & 0 & 0 & 0 & 0 \\ 
1 & 1 & 1 & 0 & 0 & 0 \\ 
0 & 1 & 1 & 1 & 0 & 0 \\ 
0 & 0 & 1 & 1 & 1 & 0 \\ 
0 & 0 & 0 & 1 & 1 & 1 \\ 
0 & 0 & 0 & 0 & 1 & 1
\end{pmatrix}
&
T_2 &= \frac{1}{3}
\begin{pmatrix}
0 & 0 & 0 & 0 & 0 & 1 \\ 
0 & 0 & 0 & 0 & 0 & 0 \\ 
0 & 0 & 0 & 0 & 0 & 0 \\ 
0 & 0 & 0 & 0 & 0 & 0 \\ 
0 & 0 & 0 & 0 & 0 & 0 \\ 
1 & 0 & 0 & 0 & 0 & 0
\end{pmatrix}.
\end{align*}
$$

From these expressions it is clear that $T_r=T_1+T_2^{\leftrightarrow}=S$.

<!-- --- end solution of exercise --- -->







<!-- --- end exercise --- -->